This notebook was created to address issue https://github.com/girder/slicer_package_manager/issues/76

## Prerequisites

* Have access to testing instance of [slicer_package_manager](https://github.com/girder/slicer_package_manager#readme) server reachable at http://192.168.113.208:8080

* Set `SLICER_EXTENSION_MANAGER_API_KEY` env variable

## Checkout description files

In [40]:
%bash

mkdir -p 35_data
if [[ ! -d 35_data/ExtensionsIndex ]]; then
  git clone git://github.com/Slicer/ExtensionsIndex 35_data/ExtensionsIndex
fi

# Download missing Sequences.s4ext
curl -L# \
  https://raw.githubusercontent.com/Slicer/ExtensionsIndex/1d7b5bf332adb4a4e8c2b17efbdca42d143b939c/Sequences.s4ext \
  -o 35_data/ExtensionsIndex/ARCHIVE/Sequences.s4ext

SyntaxError: invalid syntax (<ipython-input-40-b5b61cdc9400>, line 3)

## Retrieve extension metadata from ExtensionsIndex files

In [2]:
# Copied from  https://github.com/jcfr/pys4ext/blob/master/slicer_extensions_index_checkout.py
def parse_s4ext(ext_file_path):
    """Parse a Slicer extension description file.
    :param ext_file_path: Path to a Slicer extension description file.
    :return: Dictionnary of extension metadata.
    """
    ext_metadata = {}
    with open(ext_file_path) as ext_file:
        for line in ext_file:
            if not line.strip() or line.startswith("#"):
                continue
            fields = [field.strip() for field in line.split(' ', 1)]
            assert(len(fields) <= 2)
            ext_metadata[fields[0]] = fields[1] if len(fields) == 2 else None
    return ext_metadata

In [3]:
import glob, os
extensions_index_dir = '35_data/ExtensionsIndex'
all_s4ext_metadata = {
    os.path.basename(s4ext).split(".")[0]: parse_s4ext(s4ext) 
    for s4ext in glob.glob('%s/*.s4ext' % extensions_index_dir) + glob.glob('%s/ARCHIVE/*.s4ext' % extensions_index_dir)
}

In [13]:
all_s4ext_metadata['OpenAnatomy']

{'build_subdirectory': '.',
 'category': 'Segmentation',
 'contributors': 'Andras Lasso (PerkLab), Csaba Pinter (PerkLab), Michael Halle (SPL)',
 'depends': 'NA',
 'description': '3D Slicer extension for exporting Slicer scenes to use in the OpenAnatomy.org browser',
 'enabled': '1',
 'homepage': 'https://github.com/PerkLab/SlicerOpenAnatomy',
 'iconurl': 'https://raw.githubusercontent.com/PerkLab/SlicerOpenAnatomy/master/SlicerOpenAnatomy.png',
 'scm': 'git',
 'scmrevision': 'master',
 'scmurl': 'https://github.com/PerkLab/SlicerOpenAnatomy',
 'screenshoturls': 'https://raw.githubusercontent.com/PerkLab/SlicerOpenAnatomy/master/Screenshot01.png',
 'status': ''}

## Retrieve extensions from server

In [36]:
from slicer_package_manager_client import SlicerPackageClient

client = SlicerPackageClient(apiUrl='http://192.168.113.208:8080/api/v1')
client.authenticate(apiKey=os.environ['SLICER_EXTENSION_MANAGER_API_KEY'])

{'_id': '5a8baa26d7a3fb00011481a1'}

In [26]:
extensions = client.listExtension("Slicer", all=True)

In [38]:
from pprint import pprint as pp

In [27]:
pp(extensions[0])

{'_id': '5ef47e654db7a400019d623e',
 'baseParentId': '5a8c800cd7a3fb0001b37cdd',
 'baseParentType': 'collection',
 'created': '2020-06-25T10:37:25.114000+00:00',
 'creatorId': '5a8c7f58d7a3fb0001b37cd9',
 'description': '',
 'folderId': '5ef4686a4db7a400019d6102',
 'lowerName': '29177_slicermorph_macosx_amd64_41bae1a',
 'meta': {'app_id': '5aa3da37f3eb080001497b3d',
          'app_revision': '29177',
          'arch': 'amd64',
          'baseName': 'SlicerMorph',
          'category': 'IGT',
          'description': '',
          'homepage': 'https://github.com/gpernelle/iGyne',
          'icon_url': 'https://raw.github.com/gpernelle/iGyne/master/iGyne.png',
          'os': 'macosx',
          'repository_type': 'git',
          'repository_url': 'https://github.com/SlicerMorph/SlicerMorph.git',
          'revision': '41bae1a',
          'screenshots': 'https://raw.github.com/gpernelle/iGyne/master/LabelingResult.png'},
 'name': '29177_SlicerMorph_macosx_amd64_41bae1a',
 'size': 701001

In [29]:
name_to_s4ext_name = {
    'Autoscroll': 'SlicerAutoscroll',
    'BoneThicknessMapping': 'ABL-BoneThickness',
    'CleverSeg': 'CleverSegmentation',
    'KidneyStoneCalculator': 'SlicerKidneyStoneCalculator',
    'PETCPhantom': 'SlicerPETPhantomAnalysis',
    'RegistrationQA': 'RegQAExtension',
    'SlicerMorph': 'SlicerMorphExtension',
    'SlicerOpenAnatomy': 'OpenAnatomy',
    'SlicerRadiomics': 'Radiomics'
}

names_mismatch_s4ext= set()

# icon_url, category, homepage, screenshots, contributors, dependency
additional_metadata = {}

# Gather missing metadata
for extension in extensions:
    extension_metadata = extension['meta']
    extension_name = extension_metadata['baseName']
    
    s4ext_name = name_to_s4ext_name.get(extension_name, extension_name)
    
    if s4ext_name not in all_s4ext_metadata:
        names_mismatch_s4ext.add(extension_name)
        continue
    
    try:
        additional_metadata[extension_name] = {
            'icon_url': all_s4ext_metadata[s4ext_name]['iconurl'],
            'category': all_s4ext_metadata[s4ext_name]['category'],
            'homepage': all_s4ext_metadata[s4ext_name]['homepage'],
            'screenshots': all_s4ext_metadata[s4ext_name].get('screenshoturls', ''),
            'contributors': all_s4ext_metadata[s4ext_name]['contributors'],
            'dependency': all_s4ext_metadata[s4ext_name]['depends'],
        }
    except KeyError as exc:
        print(s4ext_name, exc)

In [33]:
# Loop over all extension items and add missing metadata
for extension in extensions:
    item_id = extension['_id']
    extension_metadata = extension['meta']
    extension_name = extension_metadata['baseName']
    print("extension_name [%s], item_id [%s]" % (extension_name, item_id))
    #pp(extension_metadata)
    for key in ['icon_url', 'screenshots', 'category', 'homepage']:
        if key not in extension_metadata:
            extension_metadata[key] = additional_metadata[extension_name][key]
    client.addMetadataToItem(item_id, extension_metadata)

extension_name [SlicerMorph], item_id [5ef47e654db7a400019d623e]
extension_name [RegistrationQA], item_id [5ef47d814db7a400019d623b]
extension_name [PathReconstruction], item_id [5ef47d404db7a400019d6238]
extension_name [GelDosimetryAnalysis], item_id [5ef47ce84db7a400019d6235]
extension_name [SegmentRegistration], item_id [5ef47c8a4db7a400019d6232]
extension_name [FilmDosimetryAnalysis], item_id [5ef47c784db7a400019d622f]
extension_name [SlicerRT], item_id [5ef47c274db7a400019d622c]
extension_name [QuantitativeReporting], item_id [5ef47bf64db7a400019d6229]
extension_name [SlicerJupyter], item_id [5ef47b484db7a400019d6226]
extension_name [SlicerCervicalSpine], item_id [5ef47b474db7a400019d6223]
extension_name [TCIABrowser], item_id [5ef47ad04db7a400019d6220]
extension_name [SliceTracker], item_id [5ef47a7e4db7a400019d621d]
extension_name [MatlabBridge], item_id [5ef47a644db7a400019d621a]
extension_name [ShapeQuantifier], item_id [5ef47a204db7a400019d6217]
extension_name [SlicerCochlea]

extension_name [MatlabBridge], item_id [5ef324534db7a400019d60cd]
extension_name [SegmentEditorExtraEffects], item_id [5ef324224db7a400019d60c8]
extension_name [DiffusionQC], item_id [5ef324224db7a400019d60c7]
extension_name [SkullStripper], item_id [5ef323d24db7a400019d60c4]
extension_name [SlicerLayoutButtons], item_id [5ef323b84db7a400019d60c1]
extension_name [ChangeTracker], item_id [5ef323724db7a400019d60be]
extension_name [SlicerProstateAblation], item_id [5ef3235a4db7a400019d60bb]
extension_name [SegmentMesher], item_id [5ef323584db7a400019d60b7]
extension_name [TCIABrowser], item_id [5ef323584db7a400019d60b5]
extension_name [SlicerFab], item_id [5ef323134db7a400019d60b2]
extension_name [UKFTractography], item_id [5ef322fb4db7a400019d60af]
extension_name [CornerAnnotation], item_id [5ef322fb4db7a400019d60ac]
extension_name [CarreraSlice], item_id [5ef3223b4db7a400019d60a9]
extension_name [CleverSeg], item_id [5ef322064db7a400019d60a6]
extension_name [ScatteredTransform], item_id

extension_name [SkullStripper], item_id [5ef1de304db7a400019d5f66]
extension_name [DiffusionQC], item_id [5ef1de2e4db7a400019d5f60]
extension_name [SegmentRegistration], item_id [5ef1de2e4db7a400019d5f5e]
extension_name [ShapeVariationAnalyzer], item_id [5ef1de2e4db7a400019d5f5d]
extension_name [SlicerLayoutButtons], item_id [5ef1ddc24db7a400019d5f5a]
extension_name [UKFTractography], item_id [5ef1dd904db7a400019d5f57]
extension_name [TCIABrowser], item_id [5ef1dd724db7a400019d5f54]
extension_name [ChangeTracker], item_id [5ef1dd5b4db7a400019d5f51]
extension_name [SlicerProstateAblation], item_id [5ef1dd394db7a400019d5f4d]
extension_name [SegmentMesher], item_id [5ef1dd394db7a400019d5f4b]
extension_name [SlicerFab], item_id [5ef1dd004db7a400019d5f48]
extension_name [CornerAnnotation], item_id [5ef1dce54db7a400019d5f45]
extension_name [TOMAAT], item_id [5ef1dc704db7a400019d5f42]
extension_name [CarreraSlice], item_id [5ef1dc5f4db7a400019d5f3f]
extension_name [SkeletalRepresentation], it

extension_name [SlicerLayoutButtons], item_id [5b07dc3d79e39e0001041564]
extension_name [CornerAnnotation], item_id [5b07dc1279e39e0001041561]
extension_name [ABC], item_id [5b07dbad79e39e000104155e]
extension_name [OsteotomyPlanner], item_id [5b07db9479e39e000104155b]
extension_name [AnglePlanesExtension], item_id [5b07db3a79e39e0001041558]
extension_name [ResectionPlanner], item_id [5b07daf279e39e0001041555]
extension_name [ModelClip], item_id [5b07dac979e39e0001041552]
extension_name [OpenCAD], item_id [5b07dabb79e39e000104154f]
extension_name [SkullStripper], item_id [5b07da8c79e39e000104154c]
extension_name [PercutaneousApproachAnalysis], item_id [5b07da8b79e39e0001041549]
extension_name [LASegmenter], item_id [5b07da8a79e39e0001041546]
extension_name [OpenCVExample], item_id [5b07da6579e39e0001041543]
extension_name [SlicerDMRI], item_id [5b07da4b79e39e0001041540]
extension_name [CleaverExtension], item_id [5b07da1579e39e000104153d]
extension_name [NeedleFinder], item_id [5b07da0

extension_name [GyroGuide], item_id [5b06885679e39e00010413f1]
extension_name [MarginCalculator], item_id [5b06883579e39e00010413ee]
extension_name [IntensitySegmenter], item_id [5b06882f79e39e00010413eb]
extension_name [SlicerOpenIGTLink], item_id [5b0687f679e39e00010413e8]
extension_name [ZFrameRegistration], item_id [5b0687a979e39e00010413e5]
extension_name [SlicerAstro], item_id [5b06871779e39e00010413e2]
extension_name [LAScarSegmenter], item_id [5b06871679e39e00010413df]
extension_name [DCMQI], item_id [5b0686f879e39e00010413dc]
extension_name [ShapeVariationAnalyzer], item_id [5b0686d579e39e00010413d9]
extension_name [DSCMRIAnalysis], item_id [5b0686c879e39e00010413d6]
extension_name [DatabaseInteractor], item_id [5b06863f79e39e00010413d3]
extension_name [ImageMaker], item_id [5b06863679e39e00010413d0]
extension_name [MeshStatisticsExtension], item_id [5b06860c79e39e00010413cd]
extension_name [exStone], item_id [5b06860079e39e00010413ca]
extension_name [WindowLevelEffect], item_

...

extension_name [SequenceRegistration], item_id [5af418f379e39e0001040b5e]
extension_name [MatlabBridge], item_id [5af4189679e39e0001040b5b]
extension_name [SlicerHeart], item_id [5af4188c79e39e0001040b58]
extension_name [ShapeQuantifier], item_id [5af4186a79e39e0001040b55]
extension_name [SlicerVASST], item_id [5af4184a79e39e0001040b52]
extension_name [SlicerRT], item_id [5af4182a79e39e0001040b4f]
extension_name [BoneTextureExtension], item_id [5af417d979e39e0001040b4c]
extension_name [PETTumorSegmentation], item_id [5af417af79e39e0001040b49]
extension_name [SwissSkullStripper], item_id [5af417a679e39e0001040b46]
extension_name [mpReview], item_id [5af4179b79e39e0001040b43]
extension_name [SlicerLayoutButtons], item_id [5af4175979e39e0001040b40]
extension_name [CornerAnnotation], item_id [5af4171379e39e0001040b3d]
extension_name [OsteotomyPlanner], item_id [5af416be79e39e0001040b3a]
extension_name [ABC], item_id [5af416a979e39e0001040b37]
extension_name [AnglePlanesExtension], item_id 

extension_name [SlicerRadiomics], item_id [5af2c28079e39e00010409f1]
extension_name [NeedleFinder], item_id [5af2c27279e39e00010409ee]
extension_name [CardiacAgatstonMeasures], item_id [5af2c26a79e39e00010409e9]
extension_name [EasyClip], item_id [5af2c26a79e39e00010409e8]
extension_name [TOMAAT], item_id [5af2c26879e39e00010409e5]
extension_name [CarreraSlice], item_id [5af2c22379e39e00010409e2]
extension_name [Slicer-Wasp], item_id [5af2c20579e39e00010409df]
extension_name [SlicerDMRI], item_id [5af2c1f179e39e00010409dc]
extension_name [CurveMaker], item_id [5af2c1d279e39e00010409d9]
extension_name [PET-IndiC], item_id [5af2c1a279e39e00010409d5]
extension_name [IASEM], item_id [5af2c1a079e39e00010409d3]
extension_name [AnomalousFiltersExtension], item_id [5af2c16779e39e00010409d0]
extension_name [DebuggingTools], item_id [5af2c14979e39e00010409cd]
extension_name [TCIABrowser], item_id [5af2c10c79e39e00010409c8]
extension_name [GyroGuide], item_id [5af2c10c79e39e00010409c7]
extension_

extension_name [PETDICOMExtension], item_id [5af16b9d79e39e0001040883]
extension_name [SlicerOpenCV], item_id [5af16b9b79e39e0001040881]
extension_name [Sequences], item_id [5af16b5d79e39e000104087e]
extension_name [LongitudinalPETCT], item_id [5af16b5979e39e000104087b]
extension_name [SegmentationWizard], item_id [5af16a9a79e39e0001040878]
extension_name [SNRMeasurement], item_id [5af16a1479e39e0001040875]
extension_name [Scoliosis], item_id [5af16a1079e39e0001040872]
extension_name [Sandbox], item_id [5af169cd79e39e000104086f]
extension_name [SlicerDevelopmentToolbox], item_id [5af1699979e39e000104086c]
extension_name [ScatteredTransform], item_id [5af1694e79e39e0001040869]
extension_name [SegmentMesher], item_id [5af1694c79e39e0001040866]
extension_name [MeshToLabelMap], item_id [5af1692979e39e0001040863]
extension_name [XNATSlicer], item_id [5af168c279e39e0001040860]
extension_name [SlicerCaseIterator], item_id [5af1678979e39e000104085d]
extension_name [Autoscroll], item_id [5af167

extension_name [SegmentRegistration], item_id [5aec31d279e39e00010405d0]
extension_name [FilmDosimetryAnalysis], item_id [5aec314779e39e00010405cd]
extension_name [PETLiverUptakeMeasurement], item_id [5aec313679e39e00010405ca]
extension_name [ShapeRegressionExtension], item_id [5aec313379e39e00010405c7]
extension_name [SequenceRegistration], item_id [5aec312379e39e00010405c4]
extension_name [SegmentEditorExtraEffects], item_id [5aec30cc79e39e00010405c1]
extension_name [MatlabBridge], item_id [5aec30ba79e39e00010405be]
extension_name [ShapeQuantifier], item_id [5aec30b479e39e00010405bb]
extension_name [SlicerHeart], item_id [5aec309679e39e00010405b8]
extension_name [SlicerVASST], item_id [5aec303679e39e00010405b5]
extension_name [mpReview], item_id [5aec303279e39e00010405b2]
extension_name [SlicerRT], item_id [5aec2fc679e39e00010405af]
extension_name [PETTumorSegmentation], item_id [5aec2fb479e39e00010405ac]
extension_name [SwissSkullStripper], item_id [5aec2fb179e39e00010405a9]
extensi

extension_name [LAScarSegmenter], item_id [5ae986ee79e39e0001040463]
extension_name [ShapeVariationAnalyzer], item_id [5ae986e479e39e0001040460]
extension_name [DSCMRIAnalysis], item_id [5ae986e179e39e000104045d]
extension_name [DCMQI], item_id [5ae986c579e39e000104045a]
extension_name [DatabaseInteractor], item_id [5ae9866879e39e0001040457]
extension_name [MeshStatisticsExtension], item_id [5ae9863179e39e0001040454]
extension_name [ImageMaker], item_id [5ae9862a79e39e0001040451]
extension_name [SlicerAstro], item_id [5ae9860b79e39e000104044e]
extension_name [exStone], item_id [5ae985f579e39e000104044b]
extension_name [WindowLevelEffect], item_id [5ae9857579e39e0001040448]
extension_name [MarkupsToModel], item_id [5ae9854979e39e0001040445]
extension_name [DeepInfer], item_id [5ae9851979e39e0001040442]
extension_name [DeveloperToolsForExtensions], item_id [5ae9850f79e39e000104043f]
extension_name [ShapeRegressionExtension], item_id [5ae984fb79e39e000104043c]
extension_name [ChangeTracke